In [1]:
# coding: utf8
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops
from PIL import Image
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
def get_onehot(a, classes=10):
    m = a.shape[0]
    b = np.zeros([m, classes])
    b[np.arange(m), a.reshape(-1)] = 1
    return b

In [3]:
train_csv = pd.read_csv('./data/input/train.csv')
X_train = train_csv.iloc[:, 1:].values
y_train_label = train_csv[['label']].values
y_train = get_onehot(y_train_label)

test_csv = pd.read_csv('./data/input/test.csv')
X_test = train_csv.values

In [4]:
imgarr = X_train[0, :].reshape([28,28])
Image.fromarray(imgarr.astype('uint8'))

In [5]:
def tf_accuracy(y, y_hat):
    is_right = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_hat, axis=1))
    is_right = tf.cast(is_right, tf.float32)
    accuracy = tf.reduce_mean(is_right)
    return accuracy

In [6]:
with tf.Session() as sess:
    a = np.array([1,2,3,4,5,6,7,8])
    b = np.array([1,3,2,4,5,7,6,8])
    
    y = tf.constant(get_onehot(a))
    y_hat = tf.constant(get_onehot(b))
    
    accuracy = tf_accuracy(y, y_hat)
    
    res = sess.run(accuracy)
    print(res)
# ans: 0.5

0.5


In [7]:
# logistics regression
def logistics_regression():
    X = tf.placeholder(tf.float32, shape=[None, 784])
    y = tf.placeholder(tf.float32, shape=[None, 10])
    
    W = tf.get_variable('W', shape=[784, 10], initializer=tf.zeros_initializer())
    b = tf.get_variable('b', shape=[1, 10], initializer=tf.zeros_initializer())
    
    z = tf.matmul(X, W) + b
    y_hat = tf.nn.softmax(z)
    
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_hat), axis=1))
#     cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z, labels=y))
    accuracy = tf_accuracy(y, y_hat)
    return X, y, cost, accuracy

In [8]:
tf.reset_default_graph()

X, y, cost, accuracy = logistics_regression()
    
with tf.Session() as sess:    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    c, a = sess.run([cost, accuracy], feed_dict={X: X_train[:5, :], y: y_train[:5, :]})
    print(c, a)

2.30259 0.4


In [51]:
# simple neural network
def simple_neural_network(layers_dim=[200, 100]):
    X = tf.placeholder(tf.float32, shape=[None, 784])
    y = tf.placeholder(tf.float32, shape=[None, 10])
    
    A = X
    hidden_layer_num = len(layers_dim)
    layers_dim = [784] + layers_dim
    for l in range(1, hidden_layer_num+1):
        W = tf.get_variable('W'+str(l), shape=[layers_dim[l-1], layers_dim[l]],
                            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable('b'+str(l), shape=[1, layers_dim[l]],
                            initializer=tf.zeros_initializer())
        Z = tf.matmul(A, W) + b
        A = tf.nn.relu(Z)
        
    W = tf.get_variable('W_out', shape=[layers_dim[-1], 10], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable('b_out', shape=[1, 10], initializer=tf.zeros_initializer())
    Z = tf.matmul(A, W) + b
    y_hat = tf.nn.softmax(Z)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z, labels=y))
    accuracy = tf_accuracy(y, y_hat)
    
    return X, y, cost, accuracy
        

In [52]:
tf.reset_default_graph()

X, y, cost, accuracy = simple_neural_network()
    
with tf.Session() as sess:    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    c, a = sess.run([cost, accuracy], feed_dict={X: X_train[:5, :], y: y_train[:5, :]})
    print(c, a)

99.6657 0.2


In [9]:
def random_mini_batches(X, y, mini_batch_size):
    m = X.shape[0]
    batch_num = np.ceil(m / mini_batch_size).astype(int)
    
    minibatches = []
    for i in range(batch_num):
        lb = i * mini_batch_size
        ub = (i+1) * mini_batch_size
        
        minibatches.append((X[lb:ub], y[lb:ub]))
    
    return minibatches

In [10]:
X = np.random.randn(10,3)
y = np.random.randn(10,1)

minibatches = random_mini_batches(X, y, 3)
for mini_X, mini_y in minibatches:
    print(mini_X.shape, mini_y.shape)

(3, 3) (3, 1)
(3, 3) (3, 1)
(3, 3) (3, 1)
(1, 3) (1, 1)


In [49]:
def train_model(X_train, y_train, model_ele=logistics_regression(), mini_batch=64, epoch=10, learning_rate=0.001):
    X_train = X_train / 255 # norm
    
    X, y, cost, accuracy = model_ele
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    m = X_train.shape[0]
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for iepoch in range(epoch):
            minibatches = random_mini_batches(X_train, y_train, mini_batch)
            tot_cost = 0
            tot_acc = 0
            
            for mini_X, mini_y in minibatches:
                res_op, res_c, res_a = sess.run([optimizer, cost, accuracy], feed_dict={X: mini_X, y: mini_y})
                
                mini_m = mini_X.shape[0]
                tot_cost += mini_m * res_c
                tot_acc += mini_m * res_a
                
            print('epoch %i, cost=%f, accuracy=%f' % (iepoch, tot_cost/m, tot_acc/m))
    

In [47]:
tf.reset_default_graph()
train_model(X_train, y_train, logistics_regression())

(<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'Mean:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1:0' shape=() dtype=float32>)
epoch 0, cost=0.610534, accuracy=0.860048
epoch 1, cost=0.341638, accuracy=0.906643
epoch 2, cost=0.304788, accuracy=0.915000
epoch 3, cost=0.287031, accuracy=0.919976
epoch 4, cost=0.276027, accuracy=0.922429
epoch 5, cost=0.268293, accuracy=0.924357
epoch 6, cost=0.262429, accuracy=0.925595
epoch 7, cost=0.257755, accuracy=0.926500
epoch 8, cost=0.253896, accuracy=0.927905
epoch 9, cost=0.250627, accuracy=0.928976


In [53]:
tf.reset_default_graph()
train_model(X_train, y_train, simple_neural_network([500, 200]))

epoch 0, cost=0.235220, accuracy=0.928786
epoch 1, cost=0.091272, accuracy=0.971905
epoch 2, cost=0.053240, accuracy=0.983690
epoch 3, cost=0.032356, accuracy=0.990238
epoch 4, cost=0.028288, accuracy=0.990833
epoch 5, cost=0.024626, accuracy=0.991500
epoch 6, cost=0.022758, accuracy=0.991762
epoch 7, cost=0.018976, accuracy=0.993429
epoch 8, cost=0.014816, accuracy=0.995048
epoch 9, cost=0.012124, accuracy=0.996000
